In [ ]:
#pip install norse

In [31]:
import torch
import torch.nn as nn
import norse.torch as norse
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer  # Thêm dòng này
from sklearn.model_selection import train_test_split

1. Đọc dữ liệu từ file CSV

In [32]:
file_path_csv = "../../data/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/ptbxl_database.csv"
df = pd.read_csv(file_path_csv)  # Thay bằng đường dẫn thực tế
image_paths = df['images'].values  # Cột chứa đường dẫn ảnh spectrogram
scp_codes = df['scp_codes'].apply(eval).values  # Chuyển chuỗi dictionary thành dict

# Chuyển SCP codes thành danh sách nhãn
scp_labels = [list(scp.keys()) for scp in scp_codes]

# One-hot encode nhãn SCP
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(scp_labels)

# Chia dữ liệu thành tập huấn luyện và kiểm tra
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, y, test_size=0.2, random_state=42
)

2. Dataset và DataLoader

In [33]:
class SpectrogramDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.float32)

# Transform cho ảnh
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Tạo dataset và dataloader
train_dataset = SpectrogramDataset(train_paths, train_labels, transform=transform)
test_dataset = SpectrogramDataset(test_paths, test_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


3. Định nghĩa mô hình SNN

In [ ]:
class SpikingNN(nn.Module):
    def __init__(self, input_shape, num_classes):
        super(SpikingNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.lif1 = norse.LIFCell()  
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.lif2 = norse.LIFCell()
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 64 * 64, 128)  
        self.lif3 = norse.LIFCell()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        # Tầng tích chập 1
        x = self.conv1(x)
        z, s = self.lif1(x)  
        x = self.pool(z)

        # Tầng tích chập 2
        x = self.conv2(x)
        z, s = self.lif2(x)  
        x = self.pool(z)

        # Flatten
        x = x.view(x.size(0), -1)

        # Fully connected layer 1
        x = self.fc1(x)
        z, s = self.lif3(x)  

        # Fully connected layer 2 (output)
        x = self.fc2(z)
        return x

4. Khởi tạo mô hình

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_shape = (256, 256)  
num_classes = len(mlb.classes_) 

model = SpikingNN(input_shape=input_shape, num_classes=num_classes).to(device)


5. Định nghĩa hàm mất mát và tối ưu hóa

In [ ]:
criterion = nn.BCEWithLogitsLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


6. Huấn luyện mô hình

In [ ]:
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")


Epoch 1/10, Loss: 0.5835
Epoch 2/10, Loss: 0.2952
Epoch 3/10, Loss: 0.1075
Epoch 4/10, Loss: 0.1041
Epoch 5/10, Loss: 0.0995
Epoch 6/10, Loss: 0.0967
Epoch 7/10, Loss: 0.0947
Epoch 8/10, Loss: 0.0923
Epoch 9/10, Loss: 0.0902
Epoch 10/10, Loss: 0.0885


7. Đánh giá mô hình

In [ ]:
model.eval()

TP = 0
TN = 0
FP = 0
FN = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        predicted = torch.sigmoid(outputs) > 0.5  

        TP += ((predicted == 1) & (labels == 1)).sum().item()
        TN += ((predicted == 0) & (labels == 0)).sum().item()
        FP += ((predicted == 1) & (labels == 0)).sum().item()
        FN += ((predicted == 0) & (labels == 1)).sum().item()

accuracy = (TP + TN) / (TP + TN + FP + FN)
sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
specificity = TN / (TN + FP) if (TN + FP) > 0 else 0

print(f"Accuracy: {accuracy:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Specificity: {specificity:.4f}")

Accuracy: 0.9713
Sensitivity (Recall): 0.3629
Precision: 0.8006
Specificity: 0.9963
